# Tackpoint

## Introduction

<img src="https://github.com/Lucmeister55/STMO-assignment/blob/main/images/sailing_intro.png?raw=true" width="600" height="600">

In the world of sailing, races are an esteemed test of skill. Both long and short distance races require careful planning in advance. In the most typical and basic form, the route planning problem requires reaching a windward mark in minimum time, or to simply pass within a certain distance of each marker, indicating the course's track.

One of the central paradigms of sailing is that boats powered solely by wind power cannot sail directly into the wind, necessitating the boat's course to alternate between headings when needing to reach an upwind destination. This maneuver, commonly known as 'tacking', is frequently employed by sailors to make progress against the wind. In the strictest meaning of sailing jargon, tacking refers to turning into the wind, whereas "gybing" indicates a more tumultuous turn away from the wind.

During a tack, the sailor typically aims to position the sailboat as close to the wind as possible while still ensuring the sails catch the wind in a way that generates aerodynamic lift, propelling the boat forward. Following this, the boat is gradually turned away from the wind to generate more lift on the sails, enabling it to move with greater speed but at a less direct path towards the destination. The range of headings that do not produce significant lift is referred to as the 'no-go zone'.

Deriving the optimal heading for any given racecourse is not trivial, even for experienced sailors. The relationship between the wind-heading angle and the sailboat's speed over ground is non-linear, hence the classical reliance on a sailor's intuition instead of simple trigonometry in such matters. Furthermore, every tack imposes an inherent speed penalty due to the sudden drop in momentum, incentivizing a minimal number of tacks to reach a destination as fast as possible. Otherwise, one could feasibly continuously tack in a way that approximates a straight line towards the marker.

It should be clear by now that the underlying physics governing the optimal course for a sailboat under specific conditions are complex and dynamic, making it challenging to intuitively determine the path. In the last decades, the sport of sailing has experienced an increasing impact of new technologies, and notably of scientiﬁc computing. Among all computational problems relevant for sailing, we are interested here in route planning and race strategy, i.e., the optimization of the yacht route.

This project aims to develop algorithms that tackle the optimization problem of determining the most efficient path for a sailboat, given a starting point, destination, and static wind conditions. Having algorithms that can generate a path plan resulting in the shortest completion time for the course can be immensely beneficial, and may diverge from expectations set even by expert sailors.

## Methods

### VMG

![VMG](https://github.com/Lucmeister55/STMO-assignment/blob/main/images/vmg.jpg?raw=true)

**Velocity made good (VMG)** is a term used in sailing, especially in yacht racing, indicating the speed of a sailboat towards (or from) the direction of the wind. Instead of sailing directly toward a windward mark, the helmsman chooses a point of sail towards the direction of the wind that maximizes VMG, representing the optimum angle against the wind. At the optimum boat speed and angle to the wind, VMG is maximized. Steering closer to the direction of the wind will reduce boat speed, while steering further away from the direction of the wind might give a higher boat speed, but at the cost of a larger deviation in heading, so less progress towards a mark.

The VMG can be calculated by using the following expression where $V_{true}$ is the velocity of the sailboat with respect to stationary ground and $\theta_s$ is the angle between current heading and the direction to destination. 

$$VMG=V_{true}∗\cos(\theta_s)$$

However, the relationship between the true velocity of the sailboat and the true wind velocity depends on what assumptions are made. Some instances take into consideration the fact that a sailboat’s Speed Over Ground increases or decreases relative to the wind direction. In theory, a sailboat’s speed increases while sailing from upwind to a downwind direction. 

Other parameters to factor in include the sail boat’s specifics that depend on the make and design of the boat. These are the 'Velocity Increase Constant', 'No – Go Zone' and 'Degree Interval', which are normally provided by the manufacturer. The physical meaning of this constant expresses 'the sail boat increases speed by 10% for every X degrees from the wind'. Considering these factors, a True VMG can be calculated using the equations:

For Upwind: 

$$VMG=\frac{V_w}{\cos(\theta_0)}*(1+\beta)^{\frac{|\theta_0-\theta_\gamma|}{i}}*\cos(\theta_s)$$

For Downwind: 

$$VMG=\frac{V_w}{\cos(\theta_0)}*(1+\beta)^{\frac{|180°-\theta_0-\theta_\gamma|}{i}}*\cos(\theta_s)$$

VMG = Velocity Made Good towards destination\
$V_w$ = Velocity of wind\
$\theta_s$ = Angle between heading and destination\
$\theta_0$ = No-go zone\
$\theta_\gamma$ = Angle between wind direction and heading\
$\beta$ = Velocity increase constant\
i = Degree Interval

In the expressions shown above the no-go zone ($\theta_0$), Degree Interval (i) and Velocity Increase Constant ($\beta$) are usually provided by the manufacturer. As these are specific to the boat’s design, commonly quoted values are used throughout this project.

### Assumptions

In order to simplify the optimization problem, the following assumptions are made and incorporated into the models:

- In this project, only relatively short-range courses are evaluated, where we thus assume that a single tack will suffice to avoid most obstacles and to reach the (next) marker in reasonable time. This is mostly to limit the scale and computational burden, but the resulting insights can nevertheless be scaled up to an extent.
- Effects of manoeuvres on the momentum of the sailboat are not considered. For example, tacking causes the sailboat to lose momentum due to the associated drag and loss of lift during the procedure. The loss of momentum is a transient process and is not modelled for this project. Hence, single tack routes are used to avoid misleading results.
- True velocity of boat at any point of time is a function of the velocity of wind and heading relative to wind direction only. This implies that at any point these are the only two variables required to calculate how fast the boat will be moving with respect to ground (true velocity) in the direction it is moving.
- Effects of momentum are not considered. As the expressions shown earlier allow for the calculation of velocity instantaneously, the time-dynamic effects are not modelled. This means that acceleration and deceleration are not taken into account. Thus, if the boat moves from a current position to the next position, the momentum is not carried or lost but the position’s assigned momentum is taken up.
- Effects of drag are not considered. The effects of drag considered are only those specified by the velocity increase constant and the no-go zone, both provided by the manufacturer. In reality, this phenomenon will have significant effects limiting the maximum velocity of the sailboat.
- We assume that no external course modifiers are applied. The tide and the force that the wind exerts on the boat will (among others) influence the boat's true trajectory by "pushing" the boat relative to it's heading, resulting in so-called leeway. Over long distances, this must be accounted for by compensating the boat's heading accordingly. Given assumption 1, we may safely consider this effect negligible over such short distances.

## Implementation

### Model
The **Single Tack Method (STM)** was developed to ensure that the path generated by the model is highly navigable. Its primary objective is to allow the algorithm to produce a path between two points with a maximum of one tack, thereby simplifying the journey to a two-leg voyage. The algorithm is tasked with determining the optimal tack length and angle that minimize the elapsed time.

It is crucial to note that the model incorporates navigational ease by restricting the number of tacks. As the number of tacks increases, the complexity of the solution process also escalates.

To illustrate this concept, let's consider a scenario where location A serves as the starting point and destination B is set. In order to create a path that permits only one tack, the problem is simplified by identifying a tack point, Ts, where the boat's travel time from A to Ts and Ts to B is less than the time it would take to sail directly from A to T and T to B. Here, T represents an arbitrary tack location. If two tacks were allowed, the algorithm would need to find the optimal combination of two points that result in the shortest elapsed time. However, due to the complex interdependence of the problem, navigating through two dimensions to identify the best combination becomes exceedingly difficult and falls outside the scope of this project.

### Objective

The objective of the algorithm is to find the path of least resistance, or the path that takes the least amount of time to complete a given course. The time taken is calculated in a discretized manner: The velocity of the sailboat is found for each step and is used to divide the step size.

$$ \text{time step k} = \frac{\text{step size}}{\text{velocity in direction of step}} $$

The time steps are added for the duration of the course in order to register the total time taken.

$$ \text{time elapsed} = \sum{\text{time step k}} $$

The objective is to optimize the path to achieve the lowest time elapsed value for any given course.

This objective function is implemented as the function *pathtime*. To deal with rounding errors causing the input of the standard library *arccos* function to lie outside the [-1, 1] domain by chance, an additional function *my_acos* was written, which removes decimals from any approximations of 1 or -1 given as input.

### Constraints
The path optimization problem in question has only one constraint: the boat must never bear a heading into the no-go zone at any given moment. However, in real-life scenarios, during a tack, the boat temporarily faces the no-go zone before regaining lift in the sails. It is the boat's momentum that enables it to steer away from the no-go zone and resume forward motion. It is important to note that the models presented in the report do not incorporate the boat's momentum terms. Consequently, if the algorithm produces a path that involves a heading into the no-go zone, the boat loses its velocity and cannot proceed further. To address this issue, the algorithms overcome this flaw by constantly restricting the heading from entering the no-go zone. This is additionally verified using the 'constrain_satisfied' function within the minimization algorithms to make absolutely sure that the convergence path stays in the feasible zone for every iteration (see further).

When travelling upwind,

$$ \theta_w > \theta_0 $$

$ \theta_0 $ = no-go zone \
$ \theta_w $ = heading relative to wind direction

### Minimization Algorithms

A (brute force) line search method is first used (*tackpoint_LS*), followed by gradient descent (*tackpoint_GD*) and Newton's method (*tackpoint_NM*) for performance comparison.

For the line search, the starting point and the current destination is specified. The model employs a pattern search algorithm incorporating an accelerating/decelerating step size. The pattern search algorithm requires to start in the feasible region because starting in the no-go zone results in an infinite path time value.

Once, the pattern search algorithm is initialised, the path time is calculated in each of the probe directions (x+, y+, x-, y-) using the function handle pathtime. This function handle accepts the starting point of the sailboat, the tack point and the destination point and integrates along the two legs of the journey to find the time elapsed (or the path time). This is the objective function value that needs to be minimised. 

With the objective function values from all four probes, the pattern search algorithm chooses the directions in x and y that favour the minimizing of the path time and determines the new tack point. Depending on the search directions (in X and Y) recorded, the pattern search algorithm updates the acceleration terms to reduce the number of pattern moves required to achieve convergence. Once convergence is reached, the optimal tack point (xt) and the optimal time elapsed values are returned.

Gradient Descent uses the *grad_f* function to calculate the gradient of the objective function, while Newton's method requires and extra function *hess_f* to compute the hessian. Both algorithms take an adaptive step in the direction of the negative gradient. If the new point is in the infeasible region, the search direction is reversed. Convergence criteria and maximum iterations are further enforced, with the latter being tunable by the user.